In [1]:
# Variabelen met meer correlatie verwijderd 

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.metrics import classification_report, roc_auc_score, plot_confusion_matrix, make_scorer, confusion_matrix
from sklearn.model_selection import GridSearchCV, StratifiedKFold, RandomizedSearchCV,StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from scipy.stats import randint
from sklearn.linear_model import SGDClassifier

from sklearn.preprocessing import OneHotEncoder



In [3]:
df_people_dataset = pd.read_csv('C:\\Users\\Mhell\\Downloads\\Thesis data\\people (1).csv.gz', encoding='ISO-8859-1')
df3_location_dataset = pd.read_csv('C:\\Users\\Mhell\\Downloads\\Thesis data\\locations.csv.gz', encoding='ISO-8859-1')
df4_connect_dataset = pd.read_csv('C:\\Users\\Mhell\\Downloads\\Thesis data\\connect.csv.gz', encoding='ISO-8859-1')

C:\Users\Mhell\AppData\Local\Temp\ipykernel_4884\1274915200.py:1: DtypeWarning: Columns (9,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df_people_dataset = pd.read_csv('C:\\Users\\Mhell\\Downloads\\Thesis data\\people (1).csv.gz', encoding='ISO-8859-1')


In [4]:
# First we filter the instances who don't represent an collaboration and are an open end in the tree (pid2 is 0)
# we do this in df4: the 'connect' dataset containing collaborations between academici.
filtered_df = df4_connect_dataset[df4_connect_dataset['pid2'] != 0]

# Then we count the amount of chilren every PID1 has. We do this by counting how often they are in the PID1 column
# since every collaboartion represents a different instance.
frequentie = filtered_df['pid1'].value_counts()

# We counted the amount of children (fecundity) and safe it in a new variable: 'fecundity'
df4_connect_dataset['fecundity'] = df4_connect_dataset['pid1'].map(frequentie)

In [5]:
# In the 'people' dataset, containing all the individual researchers (represented by a personal indintificator (pid))
# we change pid to pid1 and connect the fecundity to this pid. After that, we add 'fecundity' to the connect dataset.
df_people_dataset = df_people_dataset.rename(columns={'pid': 'pid1'})
df_people_dataset = df_people_dataset.merge(df4_connect_dataset[['pid1', 'fecundity']], on='pid1', how='left')

# we change the name of the column back to the original (in the people dataset) and delete duplicates.
df_people_dataset = df_people_dataset.rename(columns={'pid1': 'pid'})
df_people_dataset = df_people_dataset.drop_duplicates(subset='pid', keep='first')

In [6]:
# we drop the unimportant columns in the connect dataset because we are using this dataset as the basis for the merged dataset
df4_connect_dataset.drop(['addedby', 'dateadded'], axis=1, inplace=True)
df_people_dataset.drop(['firstname', 'middlename', 'addedby','homepage', 'orcid_id', 's2id', 'dateadded','area','award','lastname'], axis=1, inplace=True)

In [7]:
# we create a subset with the relevant location feature from the location dataset
locations_subset = df3_location_dataset[['country', 'locid']]

# here we add the location subset (locid and country) to the people dataset. So we add an country to every location id.
df_people_dataset = pd.merge(df_people_dataset, locations_subset, on='locid', how='left')

In [8]:
# In this codeblock we link 'locid', 'country', 'majorarea', 'degrees', 'hindex' and 'fecundity' to both collaborators
# first we add this details to the first collaborator (PID1) and then to the seconf (PID2)
# we have to rename the column because we merge the dataset based on the pid from the people dataset.

df_people_dataset_subset = df_people_dataset[['pid','locid', 'country', 'majorarea', 'degrees', 'hindex', 'fecundity']]

df4_connect_dataset.rename(columns={'pid1': 'pid'}, inplace=True)

#merge
df4_connect_dataset_pid1 = pd.merge(df4_connect_dataset, df_people_dataset_subset, on='pid', how='left')

df4_connect_dataset_pid1.rename(columns={'pid': 'pid1'}, inplace=True)
df4_connect_dataset_pid1.rename(columns={'degrees': 'degrees1'}, inplace=True)
df4_connect_dataset_pid1.rename(columns={'locid_x': 'locid_cid'}, inplace=True)
df4_connect_dataset_pid1.rename(columns={'locid_y': 'locid1'}, inplace=True)
df4_connect_dataset_pid1.rename(columns={'majorarea': 'majorarea1'}, inplace=True)
df4_connect_dataset_pid1.rename(columns={'hindex': 'hindex1'}, inplace=True)
df4_connect_dataset_pid1.rename(columns={'country': 'country1'}, inplace=True)
df4_connect_dataset_pid1.rename(columns={'fecundity': 'fecundity1'}, inplace=True)

# and now for PID2
df4_connect_dataset_pid1.rename(columns={'pid2': 'pid'}, inplace=True)

df_people_dataset_subset2 = df_people_dataset[['pid','locid', 'country', 'majorarea', 'degrees', 'hindex', 'fecundity']]

merged_pid1_and_pid2 = pd.merge(df4_connect_dataset_pid1, df_people_dataset_subset2, on='pid', how='left')

merged_pid1_and_pid2.rename(columns={'pid': 'pid2'}, inplace=True)
merged_pid1_and_pid2.rename(columns={'degrees': 'degrees2'}, inplace=True)
merged_pid1_and_pid2.rename(columns={'locid': 'locid2'}, inplace=True)
merged_pid1_and_pid2.rename(columns={'majorarea': 'majorarea2'}, inplace=True)
merged_pid1_and_pid2.rename(columns={'hindex': 'hindex2'}, inplace=True)
merged_pid1_and_pid2.rename(columns={'country': 'country2'}, inplace=True)
merged_pid1_and_pid2.rename(columns={'fecundity': 'fecundity2'}, inplace=True)

thesis_dataset = merged_pid1_and_pid2

In [9]:
# rename fecundity
thesis_dataset.drop(columns=['fecundity_y'], inplace=True)
thesis_dataset.rename(columns={'fecundity_x': 'fecundity1'}, inplace=True)

In [10]:
# deleting the non-collaborations/open ends in the tree (PID2 is 0)
thesis_dataset = thesis_dataset[thesis_dataset['pid2'] != 0]

# deleting the collaborations who does not include two countries
thesis_dataset = thesis_dataset[thesis_dataset['country1'].notna() & thesis_dataset['country2'].notna()]

# fill up all the fecundity2 with NA, by 0 (because NA means they do not have children and their fecundity is 0)
thesis_dataset['fecundity2']=thesis_dataset['fecundity2'].fillna(0)

In [11]:
# clean the countries
%run cleaning.ipynb
thesis_dataset = clean_countries(thesis_dataset)

In [12]:
# Here we create a new variable: region. We map the countries into regions.

# 1 is North America
# 2 is Europe
# 3 is Asia
# 4 is Africa
# 5 is South America-Carribean-Central America
# 6 is Middle East and North Africa
# 7 is Oceania


region_mapping = {
    'United States': 1,
    'Canada': 1,
    'United Kingdom': 2,
    'Germany': 2,
    'Netherlands': 2,
    'France': 2,
    'Hong Kong': 3,
    'Australia': 7,
    'Switzerland': 2,
    'Israel': 6,
    'China': 3,
    'Japan': 3,
    'Sweden': 2,
    'Spain': 2,
    'Brazil': 5,
    'Italy': 2,
    'India': 3,
    'South Korea': 3,
    'Belgium': 2,
    'Austria': 2,
    'Hungary': 2,
    'Denmark': 2,
    'Singapore': 3,
    'New Zealand': 7,
    'Taiwan': 3,
    'Finland': 2,
    'Mexico': 5,
    'Greece': 2,
    'Ireland': 2,
    'Norway': 2,
    'Argentina': 5,
    'Russia': 3,
    'Chile': 5,
    'Turkey': 6,
    'Poland': 2,
    'Portugal': 2,
    'South Africa':4,
    'Czechia': 2,
    'Puerto Rico': 5,
    'Iran': 6,
    'Colombia': 5,
    'Thailand': 3,
    'Iceland': 2,
    'Cyprus': 2,
    'Slovakia': 2,
    'Cameroon': 4,
    'Venezuela': 5,
    'Philippines': 3,
    'Cuba': 5,
    'Korea': 3,
    'Ukraine': 2,
    'Estonia': 2,
    'Guam': 3,
    'Kuwait': 6,
    'Egypt': 6,
    'Uruguay': 5,
    'Bangladesh': 3,
    'Croatia': 2,
    'Indonesia': 3,
    'Romania': 2,
    'Malaysia': 3,
    'Slovenia': 2,
    'Lebanon': 6,
    'U.S. Virgin Islands': 5,
    'Panama': 5,
    'United Arab Emirates': 6,
    'Tunisia': 6,
    'Macedonia (FYROM)': 2,
    'Grenada': 5,
    'Vatican City': 2,
    'Georgia': 6,
    'Jordan': 6,
    'Nepal': 3,
    'Jamaica': 5,
    'Liberia': 4,
    'The Gambia': 4,
    'Armenia': 6,
    'Costa Rica': 5,
    'Uganda': 4,
    'Bulgaria': 2,
    'Kyrgyzstan': 3,
}

thesis_dataset['region1'] = thesis_dataset['country1'].map(region_mapping)
thesis_dataset['region2'] = thesis_dataset['country2'].map(region_mapping)

In [13]:
# Then we create another variable based on membership of the OECD

OECD_mapping = {
    'United States': 1,
    'Canada': 1,
    'United Kingdom': 1,
    'Germany': 1,
    'Netherlands': 1,
    'France': 1,
    'Hong Kong': 0,
    'Australia': 1,
    'Switzerland': 1,
    'Israel': 1,
    'China': 0,
    'Japan': 1,
    'Sweden': 1,
    'Spain': 1,
    'Brazil': 0,
    'Italy': 1,
    'India': 0,
    'South Korea': 0,
    'Belgium': 1,
    'Austria': 1,
    'Hungary': 1,
    'Denmark': 1,
    'Singapore': 0,
    'New Zealand': 1,
    'Taiwan': 0,
    'Finland': 1,
    'Mexico': 1,
    'Greece': 1,
    'Ireland': 1,
    'Norway': 1,
    'Argentina': 0,
    'Russia': 0,
    'Chile': 1,
    'Turkey': 1,
    'Poland': 1,
    'Portugal': 1,
    'South Africa':0,
    'Czechia': 1,
    'Puerto Rico': 0,
    'Iran': 0,
    'Colombia': 1,   
    'Thailand': 0,
    'Iceland': 1,
    'Cyprus': 0,
    'Slovakia': 1,
    'Cameroon': 0,
    'Venezuela': 0,
    'Philippines': 0,
    'Cuba': 0,
    'Korea': 0,
    'Ukraine': 0,
    'Estonia': 1,
    'Guam': 0,
    'Kuwait': 0,
    'Egypt': 0,
    'Uruguay': 0,
    'Bangladesh': 0,
    'Croatia': 0,
    'Indonesia': 0,
    'Romania': 0,
    'Malaysia': 0,
    'Slovenia': 1,
    'Lebanon': 0,
    'U.S. Virgin Islands': 0,
    'Panama': 0,    
    'United Arab Emirates': 0,
    'Tunisia': 0,
    'Macedonia (FYROM)': 0,
    'Grenada': 0,
    'Vatican City': 0,
    'Georgia': 0,
    'Jordan': 0,
    'Nepal': 0,
    'Jamaica': 0,
    'Liberia': 0,
    'The Gambia': 0,
    'Armenia': 0,
    'Costa Rica': 1,
    'Uganda': 0,
    'Bulgaria': 0,
    'Kyrgyzstan':0,
}

thesis_dataset['OECD1'] = thesis_dataset['country1'].map(OECD_mapping)
thesis_dataset['OECD2'] = thesis_dataset['country2'].map(OECD_mapping)

# Drop NAs
thesis_dataset = thesis_dataset.dropna(subset=['OECD1', 'OECD2'])

In [14]:
# Now we create the target variables based on the international and interregional collaboration
thesis_dataset['International collaboration'] = np.where(thesis_dataset['country1'] == thesis_dataset['country2'], 0, 1)
thesis_dataset['Interregional collaboration'] = np.where(thesis_dataset['region1'] == thesis_dataset['region2'], 0, 1)

In [15]:
# Clean the degrees
%run cleaning.ipynb
thesis_dataset = clean_degrees(thesis_dataset)

In [16]:
# Encoding 'degrees' into three categories 
categories_degrees = ['PhD', 'Ed.D.']

def encode_category(degree):
    if degree in categories_degrees:
        return degree  
    else:
        return 'Other'

thesis_dataset['degrees1'] = thesis_dataset['degrees1'].apply(encode_category)
thesis_dataset['degrees2'] = thesis_dataset['degrees2'].apply(encode_category)

In [17]:
# Encoding 'fecundity' into catagories to make it more likely there is overlap between both collaborators 
# 0:'0', 1:'1-5', 2:'6-10', 3:'11-15', 4:'16-20', 5:'21+'

category_edge = [(-1), 0, 5, 10, 15, 20, float ('inf')]
category_labels = [0, 1, 2, 3, 4, 5]

thesis_dataset['fecun_category1'] = pd.cut(thesis_dataset['fecundity1'], bins= category_edge, labels=category_labels).astype(int)
thesis_dataset['fecun_category2'] = pd.cut(thesis_dataset['fecundity2'], bins= category_edge, labels=category_labels).astype(int)


In [18]:
# For six elements a Jaccard similarity score is made to see how similar both collaborators are.

# Jaccard for the amounth of children the collaborators have (in categories)
thesis_dataset['jacc_fecun_cat'] = (thesis_dataset['fecun_category1'] == thesis_dataset['fecun_category2']).astype(int)

# Jaccard score for country
thesis_dataset['jacc_country'] = (thesis_dataset['country1'] == thesis_dataset['country2']).astype(int)

# Jaccard for degrees
thesis_dataset['jacc_degrees'] = (thesis_dataset['degrees1'] == thesis_dataset['degrees2']).astype(int)

# Jaccard score for region
thesis_dataset['jacc_region'] = (thesis_dataset['region1'] == thesis_dataset['region2']).astype(int)

# Jaccard score for OECD membership
thesis_dataset['jacc_OECD'] = (thesis_dataset['OECD1'] == thesis_dataset['OECD2']).astype(int)

# Jaccard: overlap for major area
def check_overlap(majorarea1, majorarea2):
    majorarea1_elements = majorarea1.split(',') 
    majorarea2_elements = majorarea2.split(',') 
    
    majorarea1_set = set(majorarea1_elements)
    majorarea2_set = set(majorarea2_elements)
    overlap_exists = len(majorarea1_set.intersection(majorarea2_set)) > 0
    
    jacc_majorarea = int(overlap_exists)
    
    return jacc_majorarea

# new column 'jacc_majorarea' 
thesis_dataset['jacc_majorarea'] = thesis_dataset.apply(lambda row: check_overlap(row['majorarea1'], row['majorarea2']), axis=1)

In [19]:
# Create the Jaccard Coefficient 
thesis_dataset['jaccard_coefficient'] = ( thesis_dataset['jacc_fecun_cat'] + thesis_dataset['jacc_country'] + thesis_dataset['jacc_degrees'] + thesis_dataset['jacc_region']+ thesis_dataset['jacc_OECD'] +thesis_dataset['jacc_majorarea']) / 6

In [20]:
# Drop some useless variables for clarity, because of an high amounth of NANs or because the information in saved in other variables
thesis_dataset.drop(['pid1', 'pid2', 'cid','location', 'locid_cid', 'startdate','stopdate' ,'country1', 'country2', 'majorarea1', 
          'majorarea2', 'hindex1', 'hindex2', 'locid1', 'locid2',], axis=1, inplace=True)

In [21]:
# Encode categorical features
columns_to_encode = ['region1', 'region2','relation', 'fecun_category1', 'fecun_category2','degrees1', 'degrees2']
thesis_dataset = pd.get_dummies(thesis_dataset, columns=columns_to_encode, prefix=columns_to_encode, drop_first=True)

In [22]:
# Scale the continuous variables
features_to_scale = ['fecundity1', 'fecundity2']

# With standard scaler
scaler = StandardScaler()
thesis_dataset[features_to_scale] = scaler.fit_transform(thesis_dataset[features_to_scale])

In [23]:
thesis_dataset['North_America_1'] = 0
conditions = (
    (thesis_dataset['region1_2.0'] == 0) &
    (thesis_dataset['region1_3.0'] == 0) &
    (thesis_dataset['region1_4.0'] == 0) &
    (thesis_dataset['region1_5.0'] == 0) &
    (thesis_dataset['region1_6.0'] == 0) &
    (thesis_dataset['region1_7.0'] == 0)
)
thesis_dataset.loc[conditions, 'North_America_1'] = 1

thesis_dataset['North_America_2'] = 0
conditions = (
    (thesis_dataset['region2_2.0'] == 0) &
    (thesis_dataset['region2_3.0'] == 0) &
    (thesis_dataset['region2_4.0'] == 0) &
    (thesis_dataset['region2_5.0'] == 0) &
    (thesis_dataset['region2_6.0'] == 0) &
    (thesis_dataset['region2_7.0'] == 0)
)
thesis_dataset.loc[conditions, 'North_America_2'] = 1

thesis_dataset['relation_0'] = 0
conditions = (
    (thesis_dataset['relation_1'] == 0) &
    (thesis_dataset['relation_2'] == 0) &
    (thesis_dataset['relation_3'] == 0) &
    (thesis_dataset['relation_4'] == 0) 
)
thesis_dataset.loc[conditions, 'relation_0'] = 1

thesis_dataset['degrees_edd1'] = 0
conditions = (
    (thesis_dataset['degrees1_Other'] == 0) &
    (thesis_dataset['degrees1_PhD'] == 0) 
)
thesis_dataset.loc[conditions, 'degrees_edd1'] = 1

thesis_dataset['degrees_edd2'] = 0
conditions = (
    (thesis_dataset['degrees2_Other'] == 0) &
    (thesis_dataset['degrees2_PhD'] == 0) 
)
thesis_dataset.loc[conditions, 'degrees_edd2'] = 1

thesis_dataset['fecun_category1_0'] = 0
conditions = (
    (thesis_dataset['fecun_category1_2'] == 0) &
    (thesis_dataset['fecun_category1_3'] == 0) &
    (thesis_dataset['fecun_category1_4'] == 0) &
    (thesis_dataset['fecun_category1_5'] == 0) 
)
thesis_dataset.loc[conditions, 'fecun_category1_0'] = 1

thesis_dataset['fecun_category2_0'] = 0
conditions = (
    (thesis_dataset['fecun_category2_1'] == 0) &
    (thesis_dataset['fecun_category2_2'] == 0) &
    (thesis_dataset['fecun_category2_3'] == 0) &
    (thesis_dataset['fecun_category2_4'] == 0) &
    (thesis_dataset['fecun_category2_5'] == 0) 
)
thesis_dataset.loc[conditions, 'fecun_category2_0'] = 1


In [24]:
thesis_dataset['fecun_category2_1']

10         1
37         1
38         1
40         1
42         1
          ..
1561361    1
1561363    1
1561365    1
1561472    1
1561473    1
Name: fecun_category2_1, Length: 331384, dtype: uint8

In [25]:
thesis_dataset.columns

Index(['fecundity1', 'fecundity2', 'OECD1', 'OECD2',
       'International collaboration', 'Interregional collaboration',
       'jacc_fecun_cat', 'jacc_country', 'jacc_degrees', 'jacc_region',
       'jacc_OECD', 'jacc_majorarea', 'jaccard_coefficient', 'region1_2.0',
       'region1_3.0', 'region1_4.0', 'region1_5.0', 'region1_6.0',
       'region1_7.0', 'region2_2.0', 'region2_3.0', 'region2_4.0',
       'region2_5.0', 'region2_6.0', 'region2_7.0', 'relation_1', 'relation_2',
       'relation_3', 'relation_4', 'fecun_category1_2', 'fecun_category1_3',
       'fecun_category1_4', 'fecun_category1_5', 'fecun_category2_1',
       'fecun_category2_2', 'fecun_category2_3', 'fecun_category2_4',
       'fecun_category2_5', 'degrees1_Other', 'degrees1_PhD', 'degrees2_Other',
       'degrees2_PhD', 'North_America_1', 'North_America_2', 'relation_0',
       'degrees_edd1', 'degrees_edd2', 'fecun_category1_0',
       'fecun_category2_0'],
      dtype='object')

In [26]:
# Splits de data voor de eerste target variabele 'International collaboration'
y1 = thesis_dataset['International collaboration']

# Splits de data gestratificeerd (behoudt de originele verdeling van de target variabele)
train1, test1 = train_test_split(thesis_dataset, test_size=0.2, stratify=y1, random_state=101)

X_train1 = train1.drop('International collaboration', axis=1)
y_train1 = train1['International collaboration']

X_test1 = test1.drop('International collaboration', axis=1)
y_test1 = test1['International collaboration']

# Splits de data voor de tweede target variabele 'Interregional collaboration'
y2 = thesis_dataset['Interregional collaboration']

# Splits de data gestratificeerd (behoudt de originele verdeling van de target variabele)
train2, test2 = train_test_split(thesis_dataset, test_size=0.2, stratify=y2, random_state=101)

X_train2 = train2.drop('Interregional collaboration', axis=1)
y_train2 = train2['Interregional collaboration']

X_test2 = test2.drop('Interregional collaboration', axis=1)
y_test2 = test2['Interregional collaboration']


In [27]:
# Based on the highest correlation, five features per target variable are deleted
X_train1.drop([ 'Interregional collaboration','jacc_country'], axis=1, inplace=True)
X_test1.drop(['Interregional collaboration','jacc_country'], axis=1, inplace=True)
X_train2.drop([ 'International collaboration', 'jacc_region'], axis=1, inplace=True)
X_test2.drop(['International collaboration','jacc_region'], axis=1, inplace=True)

In [28]:
# Placeholder for results
results = []

# Defining the target variable
y_target = 'International collaboration'

# Empty dictionaries to store subsets
X_train_subsets = {}
X_test_subsets = {}
y_train_subsets = {}
y_test_subsets = {}

# Iterating through the subsets
for i in range(1, 8):
    subset_name = f'subset_{i}'

    # Extracting features from the subset
    X_subset_i = thesis_dataset[
        (thesis_dataset['region1_2.0'] == (i == 2)) &
        (thesis_dataset['region1_3.0'] == (i == 3)) &
        (thesis_dataset['region1_4.0'] == (i == 4)) &
        (thesis_dataset['region1_5.0'] == (i == 5)) &
        (thesis_dataset['region1_6.0'] == (i == 6)) &
        (thesis_dataset['region1_7.0'] == (i == 7))
    ]

    # Extracting target variable
    y_subset_i = X_subset_i[y_target]

    # Dropping the target variable from features
    X_subset_i = X_subset_i.drop(columns=[y_target])

    # Create a StratifiedShuffleSplit
    sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

    # Get the indices for train and test sets using stratified sampling
    train_index, test_index = next(sss.split(X_subset_i, y_subset_i))

    # Splitting the data into training and testing sets
    X_train_subset_i, X_test_subset_i = X_subset_i.iloc[train_index], X_subset_i.iloc[test_index]
    y_train_subset_i, y_test_subset_i = y_subset_i.iloc[train_index], y_subset_i.iloc[test_index]

    # Store subsets in dictionaries
    X_train_subsets[subset_name] = X_train_subset_i
    X_test_subsets[subset_name] = X_test_subset_i
    y_train_subsets[subset_name] = y_train_subset_i
    y_test_subsets[subset_name] = y_test_subset_i

    # Save them
    results.append({
        'subset_name': subset_name,
        'X_train': X_train_subset_i,
        'X_test': X_test_subset_i,
        'y_train': y_train_subset_i,
        'y_test': y_test_subset_i
    })

# The subsets outside the loop
X_train_subset_1 = X_train_subsets['subset_1']
X_test_subset_1 = X_test_subsets['subset_1']
y_train_subset_1 = y_train_subsets['subset_1']
y_test_subset_1 = y_test_subsets['subset_1']

X_train_subset_2 = X_train_subsets['subset_2']
X_test_subset_2 = X_test_subsets['subset_2']
y_train_subset_2 = y_train_subsets['subset_2']
y_test_subset_2 = y_test_subsets['subset_2']

X_train_subset_3 = X_train_subsets['subset_3']
X_test_subset_3 = X_test_subsets['subset_3']
y_train_subset_3 = y_train_subsets['subset_3']
y_test_subset_3 = y_test_subsets['subset_3']

X_train_subset_4 = X_train_subsets['subset_4']
X_test_subset_4 = X_test_subsets['subset_4']
y_train_subset_4 = y_train_subsets['subset_4']
y_test_subset_4 = y_test_subsets['subset_4']

X_train_subset_5 = X_train_subsets['subset_5']
X_test_subset_5 = X_test_subsets['subset_5']
y_train_subset_5 = y_train_subsets['subset_5']
y_test_subset_5 = y_test_subsets['subset_5']

X_train_subset_6 = X_train_subsets['subset_6']
X_test_subset_6 = X_test_subsets['subset_6']
y_train_subset_6 = y_train_subsets['subset_6']
y_test_subset_6 = y_test_subsets['subset_6']

X_train_subset_7 = X_train_subsets['subset_7']
X_test_subset_7 = X_test_subsets['subset_7']
y_train_subset_7 = y_train_subsets['subset_7']
y_test_subset_7 = y_test_subsets['subset_7']


In [29]:
# other targetvariable deleted and 'jacc_country' deleted because of the overlap with the target variable.
X_train_subset_1.drop([ 'Interregional collaboration','jacc_country'], axis=1, inplace=True)
X_test_subset_1.drop(['Interregional collaboration','jacc_country'], axis=1, inplace=True)
X_train_subset_2.drop([ 'Interregional collaboration','jacc_country'], axis=1, inplace=True)
X_test_subset_2.drop(['Interregional collaboration','jacc_country'], axis=1, inplace=True)
X_train_subset_3.drop([ 'Interregional collaboration','jacc_country'], axis=1, inplace=True)
X_test_subset_3.drop(['Interregional collaboration','jacc_country'], axis=1, inplace=True)
X_train_subset_4.drop([ 'Interregional collaboration','jacc_country'], axis=1, inplace=True)
X_test_subset_4.drop(['Interregional collaboration','jacc_country'], axis=1, inplace=True)
X_train_subset_5.drop([ 'Interregional collaboration','jacc_country'], axis=1, inplace=True)
X_test_subset_5.drop(['Interregional collaboration','jacc_country'], axis=1, inplace=True)
X_train_subset_6.drop([ 'Interregional collaboration','jacc_country'], axis=1, inplace=True)
X_test_subset_6.drop(['Interregional collaboration','jacc_country'], axis=1, inplace=True)
X_train_subset_7.drop([ 'Interregional collaboration','jacc_country'], axis=1, inplace=True)
X_test_subset_7.drop(['Interregional collaboration','jacc_country'], axis=1, inplace=True)


C:\Users\Mhell\AppData\Local\Temp\ipykernel_4884\3040456052.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_subset_7.drop([ 'Interregional collaboration','jacc_country'], axis=1, inplace=True)
C:\Users\Mhell\AppData\Local\Temp\ipykernel_4884\3040456052.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_subset_7.drop(['Interregional collaboration','jacc_country'], axis=1, inplace=True)


In [30]:
# Placeholder for results
results = []

# Defining the target variable
y_target = 'Interregional collaboration'

# Empty dictionaries to store subsets
X_train_subsets_2_ = {}
X_test_subsets_2_ = {}
y_train_subsets_2_ = {}
y_test_subsets_2_ = {}

# Iterating through the subsets
for i in range(1, 8):
    subset_name = f'subset_2_{i}'

    # Extracting features from the subset
    X_subset_2_i = thesis_dataset[
        (thesis_dataset['region1_2.0'] == (i == 2)) &
        (thesis_dataset['region1_3.0'] == (i == 3)) &
        (thesis_dataset['region1_4.0'] == (i == 4)) &
        (thesis_dataset['region1_5.0'] == (i == 5)) &
        (thesis_dataset['region1_6.0'] == (i == 6)) &
        (thesis_dataset['region1_7.0'] == (i == 7))
    ]

    # Extracting target variable
    y_subset_2_i = X_subset_2_i[y_target]

    # Dropping the target variable from features
    X_subset_2_i = X_subset_2_i.drop(columns=[y_target])

    # Create a StratifiedShuffleSplit
    sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

    # Get the indices for train and test sets using stratified sampling
    train_index, test_index = next(sss.split(X_subset_2_i, y_subset_2_i))

    # Splitting the data into training and testing sets
    X_train_subset_2_i, X_test_subset_2_i = X_subset_2_i.iloc[train_index], X_subset_2_i.iloc[test_index]
    y_train_subset_2_i, y_test_subset_2_i = y_subset_2_i.iloc[train_index], y_subset_2_i.iloc[test_index]

    # Store subsets in dictionaries
    X_train_subsets_2_[subset_name] = X_train_subset_2_i
    X_test_subsets_2_[subset_name] = X_test_subset_2_i
    y_train_subsets_2_[subset_name] = y_train_subset_2_i
    y_test_subsets_2_[subset_name] = y_test_subset_2_i

    # Save them
    results.append({
        'subset_name': subset_name,
        'X_train': X_train_subset_2_i,
        'X_test': X_test_subset_2_i,
        'y_train': y_train_subset_2_i,
        'y_test': y_test_subset_2_i
    })

# The subsets outside the loop
X_train_subset_2_1 = X_train_subsets_2_['subset_2_1']
X_test_subset_2_1 = X_test_subsets_2_['subset_2_1']
y_train_subset_2_1 = y_train_subsets_2_['subset_2_1']
y_test_subset_2_1 = y_test_subsets_2_['subset_2_1']

X_train_subset_2_2 = X_train_subsets_2_['subset_2_2']
X_test_subset_2_2 = X_test_subsets_2_['subset_2_2']
y_train_subset_2_2 = y_train_subsets_2_['subset_2_2']
y_test_subset_2_2 = y_test_subsets_2_['subset_2_2']

X_train_subset_2_3 = X_train_subsets_2_['subset_2_3']
X_test_subset_2_3 = X_test_subsets_2_['subset_2_3']
y_train_subset_2_3 = y_train_subsets_2_['subset_2_3']
y_test_subset_2_3 = y_test_subsets_2_['subset_2_3']

X_train_subset_2_4 = X_train_subsets_2_['subset_2_4']
X_test_subset_2_4 = X_test_subsets_2_['subset_2_4']
y_train_subset_2_4 = y_train_subsets_2_['subset_2_4']
y_test_subset_2_4 = y_test_subsets_2_['subset_2_4']

X_train_subset_2_5 = X_train_subsets_2_['subset_2_5']
X_test_subset_2_5 = X_test_subsets_2_['subset_2_5']
y_train_subset_2_5 = y_train_subsets_2_['subset_2_5']
y_test_subset_2_5 = y_test_subsets_2_['subset_2_5']

X_train_subset_2_6 = X_train_subsets_2_['subset_2_6']
X_test_subset_2_6 = X_test_subsets_2_['subset_2_6']
y_train_subset_2_6 = y_train_subsets_2_['subset_2_6']
y_test_subset_2_6 = y_test_subsets_2_['subset_2_6']

X_train_subset_2_7 = X_train_subsets_2_['subset_2_7']
X_test_subset_2_7 = X_test_subsets_2_['subset_2_7']
y_train_subset_2_7 = y_train_subsets_2_['subset_2_7']
y_test_subset_2_7 = y_test_subsets_2_['subset_2_7']


In [31]:
# other targetvariable deleted and 'jacc_region' deleted because of the overlap with the target variable.
X_train_subset_2_1.drop([ 'International collaboration', 'jacc_region'], axis=1, inplace=True)
X_test_subset_2_1.drop(['International collaboration','jacc_region'], axis=1, inplace=True)
X_train_subset_2_2.drop([ 'International collaboration', 'jacc_region'], axis=1, inplace=True)
X_test_subset_2_2.drop(['International collaboration','jacc_region'], axis=1, inplace=True)
X_train_subset_2_3.drop([ 'International collaboration', 'jacc_region'], axis=1, inplace=True)
X_test_subset_2_3.drop(['International collaboration','jacc_region'], axis=1, inplace=True)
X_train_subset_2_4.drop([ 'International collaboration', 'jacc_region'], axis=1, inplace=True)
X_test_subset_2_4.drop(['International collaboration','jacc_region'], axis=1, inplace=True)
X_train_subset_2_5.drop([ 'International collaboration', 'jacc_region'], axis=1, inplace=True)
X_test_subset_2_5.drop(['International collaboration','jacc_region'], axis=1, inplace=True)
X_train_subset_2_6.drop([ 'International collaboration', 'jacc_region'], axis=1, inplace=True)
X_test_subset_2_6.drop(['International collaboration','jacc_region'], axis=1, inplace=True)
X_train_subset_2_7.drop([ 'International collaboration', 'jacc_region'], axis=1, inplace=True)
X_test_subset_2_7.drop(['International collaboration','jacc_region'], axis=1, inplace=True)


C:\Users\Mhell\AppData\Local\Temp\ipykernel_4884\2286305852.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_subset_2_7.drop([ 'International collaboration', 'jacc_region'], axis=1, inplace=True)
C:\Users\Mhell\AppData\Local\Temp\ipykernel_4884\2286305852.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_subset_2_7.drop(['International collaboration','jacc_region'], axis=1, inplace=True)


In [32]:
# Feature importance per subset, everything lower than 0.001 is deleted.
# Subset 1
param_grid = {'n_estimators': [int(x) for x in range(50, 251, 50)],
    'min_samples_split': [2, 4, 6, 8, 10],
    'min_samples_leaf': [1, 2, 4, 8],
    'max_depth': [None, 10, 20, 30, 40, 50]}

# Initialize the RandomForestClassifier (use it with the best hyperparameters), RandomizedSearchCV (add it on the training data)
# Then fit the model
rf_classifier = RandomForestClassifier(random_state=42)
random_search = RandomizedSearchCV(estimator=rf_classifier, param_distributions=param_grid, n_iter=3, cv=5, random_state=42)
random_search.fit(X_train_subset_1, y_train_subset_1)
best_params = random_search.best_params_
rf_classifier = RandomForestClassifier(**best_params, random_state=42)
rf_classifier.fit(X_train_subset_1, y_train_subset_1)

# Feature importance without hyperparameter tuning
feature_importance_rf = rf_classifier.feature_importances_

# Creating a DataFrame for the feature importance
feature_importance = pd.DataFrame({'Feature': X_train_subset_1.columns, 'Importance': feature_importance_rf})
feature_importance = feature_importance.sort_values(by='Importance', ascending=False)
feature_importance['Importance'] = feature_importance['Importance'].round(6)

# Print variables with importance >= 0.001
print("Variables with importance for North America >= 0.001:")
print(feature_importance[feature_importance['Importance'] >= 0.001])
print()

# Remove variables with importance < 0.001
low_importance_variables = feature_importance[feature_importance['Importance'] < 0.001]['Feature'].tolist()
X_train_subset_1 = X_train_subset_1.drop(columns=low_importance_variables)


# Subset 2
param_grid = {'n_estimators': [int(x) for x in range(50, 251, 50)],
    'min_samples_split': [2, 4, 6, 8, 10],
    'min_samples_leaf': [1, 2, 4, 8],
    'max_depth': [None, 10, 20, 30, 40, 50]}

# Initialize the RandomForestClassifier (use it with the best hyperparameters), RandomizedSearchCV (add it on the training data)
# Then fit the model
rf_classifier = RandomForestClassifier(random_state=42)
random_search = RandomizedSearchCV(estimator=rf_classifier, param_distributions=param_grid, n_iter=3, cv=5, random_state=42)
random_search.fit(X_train_subset_2, y_train_subset_2)
best_params = random_search.best_params_
rf_classifier = RandomForestClassifier(**best_params, random_state=42)
rf_classifier.fit(X_train_subset_2, y_train_subset_2)

# Feature importance without hyperparameter tuning
feature_importance_rf = rf_classifier.feature_importances_

# Creating a DataFrame for the feature importance
feature_importance = pd.DataFrame({'Feature': X_train_subset_2.columns, 'Importance': feature_importance_rf})
feature_importance = feature_importance.sort_values(by='Importance', ascending=False)
feature_importance['Importance'] = feature_importance['Importance'].round(6)

# Print variables with importance >= 0.001
print("Variables with importance for Europe >= 0.001:")
print(feature_importance[feature_importance['Importance'] >= 0.001])
print()

# Remove variables with importance < 0.001
low_importance_variables = feature_importance[feature_importance['Importance'] < 0.001]['Feature'].tolist()
X_train_subset_2 = X_train_subset_2.drop(columns=low_importance_variables)


# Subset 3
param_grid = {'n_estimators': [int(x) for x in range(50, 251, 50)],
    'min_samples_split': [2, 4, 6, 8, 10],
    'min_samples_leaf': [1, 2, 4, 8],
    'max_depth': [None, 10, 20, 30, 40, 50]}

# Initialize the RandomForestClassifier (use it with the best hyperparameters), RandomizedSearchCV (add it on the training data)
# Then fit the model
rf_classifier = RandomForestClassifier(random_state=42)
random_search = RandomizedSearchCV(estimator=rf_classifier, param_distributions=param_grid, n_iter=3, cv=5, random_state=42)
random_search.fit(X_train_subset_3, y_train_subset_3)
best_params = random_search.best_params_
rf_classifier = RandomForestClassifier(**best_params, random_state=42)
rf_classifier.fit(X_train_subset_3, y_train_subset_3)

# Feature importance without hyperparameter tuning
feature_importance_rf = rf_classifier.feature_importances_

# Creating a DataFrame for the feature importance
feature_importance = pd.DataFrame({'Feature': X_train_subset_3.columns, 'Importance': feature_importance_rf})
feature_importance = feature_importance.sort_values(by='Importance', ascending=False)
feature_importance['Importance'] = feature_importance['Importance'].round(6)

# Print variables with importance >= 0.001
print("Variables with importance for Asia >= 0.001:")
print(feature_importance[feature_importance['Importance'] >= 0.001])

# Remove variables with importance < 0.001
low_importance_variables = feature_importance[feature_importance['Importance'] < 0.001]['Feature'].tolist()
X_train_subset_3 = X_train_subset_3.drop(columns=low_importance_variables)


# Subset 4
param_grid = {'n_estimators': [int(x) for x in range(50, 251, 50)],
    'min_samples_split': [2, 4, 6, 8, 10],
    'min_samples_leaf': [1, 2, 4, 8],
    'max_depth': [None, 10, 20, 30, 40, 50]}

# Initialize the RandomForestClassifier (use it with the best hyperparameters), RandomizedSearchCV (add it on the training data)
# Then fit the model
rf_classifier = RandomForestClassifier(random_state=42)
random_search = RandomizedSearchCV(estimator=rf_classifier, param_distributions=param_grid, n_iter=3, cv=5, random_state=42)
random_search.fit(X_train_subset_4, y_train_subset_4)
best_params = random_search.best_params_
rf_classifier = RandomForestClassifier(**best_params, random_state=42)
rf_classifier.fit(X_train_subset_4, y_train_subset_4)

# Feature importance without hyperparameter tuning
feature_importance_rf = rf_classifier.feature_importances_

# Creating a DataFrame for the feature importance
feature_importance = pd.DataFrame({'Feature': X_train_subset_4.columns, 'Importance': feature_importance_rf})
feature_importance = feature_importance.sort_values(by='Importance', ascending=False)
feature_importance['Importance'] = feature_importance['Importance'].round(6)

# Print variables with importance >= 0.001
print("Variables with importance for Africa >= 0.001:")
print(feature_importance[feature_importance['Importance'] >= 0.001])
print()

# Remove variables with importance < 0.001
low_importance_variables = feature_importance[feature_importance['Importance'] < 0.001]['Feature'].tolist()
X_train_subset_4 = X_train_subset_4.drop(columns=low_importance_variables)


# Subset 5
param_grid = {'n_estimators': [int(x) for x in range(50, 251, 50)],
    'min_samples_split': [2, 4, 6, 8, 10],
    'min_samples_leaf': [1, 2, 4, 8],
    'max_depth': [None, 10, 20, 30, 40, 50]}

# Initialize the RandomForestClassifier (use it with the best hyperparameters), RandomizedSearchCV (add it on the training data)
# Then fit the model
rf_classifier = RandomForestClassifier(random_state=42)
random_search = RandomizedSearchCV(estimator=rf_classifier, param_distributions=param_grid, n_iter=3, cv=5, random_state=42)
random_search.fit(X_train_subset_5, y_train_subset_5)
best_params = random_search.best_params_
rf_classifier = RandomForestClassifier(**best_params, random_state=42)
rf_classifier.fit(X_train_subset_5, y_train_subset_5)

# Feature importance without hyperparameter tuning
feature_importance_rf = rf_classifier.feature_importances_

# Creating a DataFrame for the feature importance
feature_importance = pd.DataFrame({'Feature': X_train_subset_5.columns, 'Importance': feature_importance_rf})
feature_importance = feature_importance.sort_values(by='Importance', ascending=False)
feature_importance['Importance'] = feature_importance['Importance'].round(6)

# Print variables with importance >= 0.001
print("Variables with importance for Latin America >= 0.001:")
print(feature_importance[feature_importance['Importance'] >= 0.001])
print()

# Remove variables with importance < 0.001
low_importance_variables = feature_importance[feature_importance['Importance'] < 0.001]['Feature'].tolist()
X_train_subset_5 = X_train_subset_5.drop(columns=low_importance_variables)


# Subset 6
param_grid = {'n_estimators': [int(x) for x in range(50, 251, 50)],
    'min_samples_split': [2, 4, 6, 8, 10],
    'min_samples_leaf': [1, 2, 4, 8],
    'max_depth': [None, 10, 20, 30, 40, 50]}

# Initialize the RandomForestClassifier (use it with the best hyperparameters), RandomizedSearchCV (add it on the training data)
# Then fit the model
rf_classifier = RandomForestClassifier(random_state=42)
random_search = RandomizedSearchCV(estimator=rf_classifier, param_distributions=param_grid, n_iter=3, cv=5, random_state=42)
random_search.fit(X_train_subset_6, y_train_subset_6)
best_params = random_search.best_params_
rf_classifier = RandomForestClassifier(**best_params, random_state=42)
rf_classifier.fit(X_train_subset_6, y_train_subset_6)

# Feature importance without hyperparameter tuning
feature_importance_rf = rf_classifier.feature_importances_

# Creating a DataFrame for the feature importance
feature_importance = pd.DataFrame({'Feature': X_train_subset_6.columns, 'Importance': feature_importance_rf})
feature_importance = feature_importance.sort_values(by='Importance', ascending=False)
feature_importance['Importance'] = feature_importance['Importance'].round(6)

# Print variables with importance >= 0.001
print("Variables with importance for Arab countries >= 0.001:")
print(feature_importance[feature_importance['Importance'] >= 0.001])
print()

# Remove variables with importance < 0.001
low_importance_variables = feature_importance[feature_importance['Importance'] < 0.001]['Feature'].tolist()
X_train_subset_6 = X_train_subset_6.drop(columns=low_importance_variables)


# Subset 7 
param_grid = {'n_estimators': [int(x) for x in range(50, 251, 50)],
    'min_samples_split': [2, 4, 6, 8, 10],
    'min_samples_leaf': [1, 2, 4, 8],
    'max_depth': [None, 10, 20, 30, 40, 50]}

# Initialize the RandomForestClassifier (use it with the best hyperparameters), RandomizedSearchCV (add it on the training data)
# Then fit the model
rf_classifier = RandomForestClassifier(random_state=42)
random_search = RandomizedSearchCV(estimator=rf_classifier, param_distributions=param_grid, n_iter=3, cv=5, random_state=42)
random_search.fit(X_train_subset_7, y_train_subset_7)
best_params = random_search.best_params_
rf_classifier = RandomForestClassifier(**best_params, random_state=42)
rf_classifier.fit(X_train_subset_7, y_train_subset_7)

# Feature importance without hyperparameter tuning
feature_importance_rf = rf_classifier.feature_importances_

# Creating a DataFrame for the feature importance
feature_importance = pd.DataFrame({'Feature': X_train_subset_7.columns, 'Importance': feature_importance_rf})
feature_importance = feature_importance.sort_values(by='Importance', ascending=False)
feature_importance['Importance'] = feature_importance['Importance'].round(6)

# Print variables with importance >= 0.001
print("Variables with importance >= 0.001:")
print(feature_importance[feature_importance['Importance'] >= 0.001])

# Remove variables with importance < 0.001
low_importance_variables = feature_importance[feature_importance['Importance'] < 0.001]['Feature'].tolist()
X_train_subset_7 = X_train_subset_7.drop(columns=low_importance_variables)


Variables with importance for North America >= 0.001:
                Feature  Importance
9   jaccard_coefficient    0.239414
40      North_America_2    0.206453
6           jacc_region    0.175656
16          region2_2.0    0.147909
8        jacc_majorarea    0.078179
0            fecundity1    0.029189
5          jacc_degrees    0.014694
3                 OECD2    0.014580
7             jacc_OECD    0.010572
17          region2_3.0    0.010387
1            fecundity2    0.010107
4        jacc_fecun_cat    0.009931
35       degrees1_Other    0.007366
21          region2_7.0    0.007170
30    fecun_category2_1    0.005868
36         degrees1_PhD    0.005576
22           relation_1    0.005346
45    fecun_category2_0    0.003588
38         degrees2_PhD    0.003491
44    fecun_category1_0    0.002674
20          region2_6.0    0.002430
37       degrees2_Other    0.001912
31    fecun_category2_2    0.001545
26    fecun_category1_2    0.001394
23           relation_2    0.001031

Variables

In [33]:
# Feature importance per subset, everything lower than 0.001 is deleted.
# Subset 1
param_grid = {'n_estimators': [int(x) for x in range(50, 251, 50)],
    'min_samples_split': [2, 4, 6, 8, 10],
    'min_samples_leaf': [1, 2, 4, 8],
    'max_depth': [None, 10, 20, 30, 40, 50]}

# Initialize the RandomForestClassifier (use it with the best hyperparameters), RandomizedSearchCV (add it on the training data)
# Then fit the model
rf_classifier = RandomForestClassifier(random_state=42)
random_search = RandomizedSearchCV(estimator=rf_classifier, param_distributions=param_grid, n_iter=3, cv=5, random_state=42)
random_search.fit(X_train_subset_2_1, y_train_subset_2_1)
best_params = random_search.best_params_
rf_classifier = RandomForestClassifier(**best_params, random_state=42)
rf_classifier.fit(X_train_subset_2_1, y_train_subset_2_1)

# Feature importance without hyperparameter tuning
feature_importance_rf = rf_classifier.feature_importances_

# Creating a DataFrame for the feature importance
feature_importance = pd.DataFrame({'Feature': X_train_subset_2_1.columns, 'Importance': feature_importance_rf})
feature_importance = feature_importance.sort_values(by='Importance', ascending=False)
feature_importance['Importance'] = feature_importance['Importance'].round(6)

# Print variables with importance >= 0.001
print("Variables with importance for North America >= 0.001:")
print(feature_importance[feature_importance['Importance'] >= 0.001])
print()

# Remove variables with importance < 0.001
low_importance_variables = feature_importance[feature_importance['Importance'] < 0.001]['Feature'].tolist()
X_train_subset_2_1 = X_train_subset_2_1.drop(columns=low_importance_variables)


# Subset 2
param_grid = {'n_estimators': [int(x) for x in range(50, 251, 50)],
    'min_samples_split': [2, 4, 6, 8, 10],
    'min_samples_leaf': [1, 2, 4, 8],
    'max_depth': [None, 10, 20, 30, 40, 50]}

# Initialize the RandomForestClassifier (use it with the best hyperparameters), RandomizedSearchCV (add it on the training data)
# Then fit the model
rf_classifier = RandomForestClassifier(random_state=42)
random_search = RandomizedSearchCV(estimator=rf_classifier, param_distributions=param_grid, n_iter=3, cv=5, random_state=42)
random_search.fit(X_train_subset_2_2, y_train_subset_2_2)
best_params = random_search.best_params_
rf_classifier = RandomForestClassifier(**best_params, random_state=42)
rf_classifier.fit(X_train_subset_2_2, y_train_subset_2_2)

# Feature importance without hyperparameter tuning
feature_importance_rf = rf_classifier.feature_importances_

# Creating a DataFrame for the feature importance
feature_importance = pd.DataFrame({'Feature': X_train_subset_2_2.columns, 'Importance': feature_importance_rf})
feature_importance = feature_importance.sort_values(by='Importance', ascending=False)
feature_importance['Importance'] = feature_importance['Importance'].round(6)

# Print variables with importance >= 0.001
print("Variables with importance for Europe >= 0.001:")
print(feature_importance[feature_importance['Importance'] >= 0.001])
print()

# Remove variables with importance < 0.001
low_importance_variables = feature_importance[feature_importance['Importance'] < 0.001]['Feature'].tolist()
X_train_subset_2_2 = X_train_subset_2_2.drop(columns=low_importance_variables)


# Subset 3
param_grid = {'n_estimators': [int(x) for x in range(50, 251, 50)],
    'min_samples_split': [2, 4, 6, 8, 10],
    'min_samples_leaf': [1, 2, 4, 8],
    'max_depth': [None, 10, 20, 30, 40, 50]}

# Initialize the RandomForestClassifier (use it with the best hyperparameters), RandomizedSearchCV (add it on the training data)
# Then fit the model
rf_classifier = RandomForestClassifier(random_state=42)
random_search = RandomizedSearchCV(estimator=rf_classifier, param_distributions=param_grid, n_iter=3, cv=5, random_state=42)
random_search.fit(X_train_subset_2_3, y_train_subset_2_3)
best_params = random_search.best_params_
rf_classifier = RandomForestClassifier(**best_params, random_state=42)
rf_classifier.fit(X_train_subset_2_3, y_train_subset_2_3)

# Feature importance without hyperparameter tuning
feature_importance_rf = rf_classifier.feature_importances_

# Creating a DataFrame for the feature importance
feature_importance = pd.DataFrame({'Feature': X_train_subset_2_3.columns, 'Importance': feature_importance_rf})
feature_importance = feature_importance.sort_values(by='Importance', ascending=False)
feature_importance['Importance'] = feature_importance['Importance'].round(6)

# Print variables with importance >= 0.001
print("Variables with importance for Asia >= 0.001:")
print(feature_importance[feature_importance['Importance'] >= 0.001])

# Remove variables with importance < 0.001
low_importance_variables = feature_importance[feature_importance['Importance'] < 0.001]['Feature'].tolist()
X_train_subset_2_3 = X_train_subset_2_3.drop(columns=low_importance_variables)


# Subset 4
param_grid = {'n_estimators': [int(x) for x in range(50, 251, 50)],
    'min_samples_split': [2, 4, 6, 8, 10],
    'min_samples_leaf': [1, 2, 4, 8],
    'max_depth': [None, 10, 20, 30, 40, 50]}

# Initialize the RandomForestClassifier (use it with the best hyperparameters), RandomizedSearchCV (add it on the training data)
# Then fit the model
rf_classifier = RandomForestClassifier(random_state=42)
random_search = RandomizedSearchCV(estimator=rf_classifier, param_distributions=param_grid, n_iter=3, cv=5, random_state=42)
random_search.fit(X_train_subset_2_4, y_train_subset_2_4)
best_params = random_search.best_params_
rf_classifier = RandomForestClassifier(**best_params, random_state=42)
rf_classifier.fit(X_train_subset_2_4, y_train_subset_2_4)

# Feature importance without hyperparameter tuning
feature_importance_rf = rf_classifier.feature_importances_

# Creating a DataFrame for the feature importance
feature_importance = pd.DataFrame({'Feature': X_train_subset_2_4.columns, 'Importance': feature_importance_rf})
feature_importance = feature_importance.sort_values(by='Importance', ascending=False)
feature_importance['Importance'] = feature_importance['Importance'].round(6)

# Print variables with importance >= 0.001
print("Variables with importance for Africa >= 0.001:")
print(feature_importance[feature_importance['Importance'] >= 0.001])
print()

# Remove variables with importance < 0.001
low_importance_variables = feature_importance[feature_importance['Importance'] < 0.001]['Feature'].tolist()
X_train_subset_2_4 = X_train_subset_2_4.drop(columns=low_importance_variables)


# Subset 5
param_grid = {'n_estimators': [int(x) for x in range(50, 251, 50)],
    'min_samples_split': [2, 4, 6, 8, 10],
    'min_samples_leaf': [1, 2, 4, 8],
    'max_depth': [None, 10, 20, 30, 40, 50]}

# Initialize the RandomForestClassifier (use it with the best hyperparameters), RandomizedSearchCV (add it on the training data)
# Then fit the model
rf_classifier = RandomForestClassifier(random_state=42)
random_search = RandomizedSearchCV(estimator=rf_classifier, param_distributions=param_grid, n_iter=3, cv=5, random_state=42)
random_search.fit(X_train_subset_2_5, y_train_subset_2_5)
best_params = random_search.best_params_
rf_classifier = RandomForestClassifier(**best_params, random_state=42)
rf_classifier.fit(X_train_subset_2_5, y_train_subset_2_5)

# Feature importance without hyperparameter tuning
feature_importance_rf = rf_classifier.feature_importances_

# Creating a DataFrame for the feature importance
feature_importance = pd.DataFrame({'Feature': X_train_subset_2_5.columns, 'Importance': feature_importance_rf})
feature_importance = feature_importance.sort_values(by='Importance', ascending=False)
feature_importance['Importance'] = feature_importance['Importance'].round(6)

# Print variables with importance >= 0.001
print("Variables with importance for Latin America >= 0.001:")
print(feature_importance[feature_importance['Importance'] >= 0.001])
print()

# Remove variables with importance < 0.001
low_importance_variables = feature_importance[feature_importance['Importance'] < 0.001]['Feature'].tolist()
X_train_subset_2_5 = X_train_subset_2_5.drop(columns=low_importance_variables)


# Subset 6
param_grid = {'n_estimators': [int(x) for x in range(50, 251, 50)],
    'min_samples_split': [2, 4, 6, 8, 10],
    'min_samples_leaf': [1, 2, 4, 8],
    'max_depth': [None, 10, 20, 30, 40, 50]}

# Initialize the RandomForestClassifier (use it with the best hyperparameters), RandomizedSearchCV (add it on the training data)
# Then fit the model
rf_classifier = RandomForestClassifier(random_state=42)
random_search = RandomizedSearchCV(estimator=rf_classifier, param_distributions=param_grid, n_iter=3, cv=5, random_state=42)
random_search.fit(X_train_subset_2_6, y_train_subset_2_6)
best_params = random_search.best_params_
rf_classifier = RandomForestClassifier(**best_params, random_state=42)
rf_classifier.fit(X_train_subset_2_6, y_train_subset_2_6)

# Feature importance without hyperparameter tuning
feature_importance_rf = rf_classifier.feature_importances_

# Creating a DataFrame for the feature importance
feature_importance = pd.DataFrame({'Feature': X_train_subset_2_6.columns, 'Importance': feature_importance_rf})
feature_importance = feature_importance.sort_values(by='Importance', ascending=False)
feature_importance['Importance'] = feature_importance['Importance'].round(6)

# Print variables with importance >= 0.001
print("Variables with importance for Arab countries >= 0.001:")
print(feature_importance[feature_importance['Importance'] >= 0.001])
print()

# Remove variables with importance < 0.001
low_importance_variables = feature_importance[feature_importance['Importance'] < 0.001]['Feature'].tolist()
X_train_subset_2_6 = X_train_subset_2_6.drop(columns=low_importance_variables)


# Subset 7 
param_grid = {'n_estimators': [int(x) for x in range(50, 251, 50)],
    'min_samples_split': [2, 4, 6, 8, 10],
    'min_samples_leaf': [1, 2, 4, 8],
    'max_depth': [None, 10, 20, 30, 40, 50]}

# Initialize the RandomForestClassifier (use it with the best hyperparameters), RandomizedSearchCV (add it on the training data)
# Then fit the model
rf_classifier = RandomForestClassifier(random_state=42)
random_search = RandomizedSearchCV(estimator=rf_classifier, param_distributions=param_grid, n_iter=3, cv=5, random_state=42)
random_search.fit(X_train_subset_2_7, y_train_subset_2_7)
best_params = random_search.best_params_
rf_classifier = RandomForestClassifier(**best_params, random_state=42)
rf_classifier.fit(X_train_subset_2_7, y_train_subset_2_7)

# Feature importance without hyperparameter tuning
feature_importance_rf = rf_classifier.feature_importances_

# Creating a DataFrame for the feature importance
feature_importance = pd.DataFrame({'Feature': X_train_subset_2_7.columns, 'Importance': feature_importance_rf})
feature_importance = feature_importance.sort_values(by='Importance', ascending=False)
feature_importance['Importance'] = feature_importance['Importance'].round(6)

# Print variables with importance >= 0.001
print("Variables with importance >= 0.001:")
print(feature_importance[feature_importance['Importance'] >= 0.001])

# Remove variables with importance < 0.001
low_importance_variables = feature_importance[feature_importance['Importance'] < 0.001]['Feature'].tolist()
X_train_subset_2_7 = X_train_subset_2_7.drop(columns=low_importance_variables)


Variables with importance for North America >= 0.001:
                Feature  Importance
40      North_America_2    0.392506
16          region2_2.0    0.212865
5          jacc_country    0.157746
9   jaccard_coefficient    0.120595
3                 OECD2    0.018295
17          region2_3.0    0.016955
7             jacc_OECD    0.016490
0            fecundity1    0.010843
21          region2_7.0    0.010776
20          region2_6.0    0.007768
6          jacc_degrees    0.005180
8        jacc_majorarea    0.004117
30    fecun_category2_1    0.004043
19          region2_5.0    0.003598
35       degrees1_Other    0.003393
4        jacc_fecun_cat    0.003190
1            fecundity2    0.002635
22           relation_1    0.002200
45    fecun_category2_0    0.001786
36         degrees1_PhD    0.001554

Variables with importance for Europe >= 0.001:
                Feature  Importance
16          region2_2.0    0.441887
40      North_America_2    0.261790
9   jaccard_coefficient    0.13313